In [ ]:
!pip install -U langchain
!pip install -U langchain_community
!pip install -U langchain_openai
!pip install -U faiss-cpu
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 1.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.document_loaders.text import TextLoader
# loading documents
loader = PyPDFLoader("/content/lollll.pdf")
pdfpages=loader.load_and_split()



from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings

import os

os.environ["OPENAI_API_KEY"]="# your key value "


In [ ]:
doc=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000 ,chunk_overlap=0)
texts=text_splitter.split_documents(doc)


embeddings = OpenAIEmbeddings()


db=FAISS.from_documents(texts , embeddings)

In [ ]:
db

In [ ]:
retriever = db.as_retriever()

from langchain.agents.agent_toolkits import create_retriever_tool

# Choose a valid function name (e.g., describe_and_search)
tool=create_retriever_tool(
  retriever,
  "describe_and_search",  # Replace with your chosen name
  "searched and returns documents "
)

tools =[tool]

from langchain.agents.agent_toolkits import create_conversational_retrieval_agent


from langchain_openai.chat_models import ChatOpenAI

llm=ChatOpenAI(temperature=0)

agent_executor = create_conversational_retrieval_agent(llm,tools, verbose=True)


In [ ]:
input="define management? "
result=agent_executor.invoke({"input": input+"Give me answer in 50 words"})



> Entering new AgentExecutor chain...
Management is the process of planning, organizing, leading, and controlling resources within an organization to achieve specific goals and objectives. It involves coordinating people, tasks, and resources effectively and efficiently to ensure the smooth operation of the business and the attainment of desired outcomes.

> Finished chain.


In [ ]:
print(result)

{'input': 'define management? Give me answer in 50 words', 'chat_history': [HumanMessage(content='define government intelligence? Give me answer in 50 words'), AIMessage(content='Government intelligence refers to the collection, analysis, and dissemination of information by government agencies to support national security and decision-making. It involves gathering data through various means, such as surveillance and espionage, to assess threats, prevent risks, and protect the interests of a country.'), HumanMessage(content='define management? Give me answer in 50 words'), AIMessage(content='Management is the process of planning, organizing, leading, and controlling resources within an organization to achieve specific goals and objectives. It involves coordinating people, tasks, and resources effectively and efficiently to ensure the smooth operation of the business and the attainment of desired outcomes.')], 'output': 'Management is the process of planning, organizing, leading, and con

In [ ]:
#lets go for the chatPrompt templete for better input to the llm
from langchain_core.prompts import ChatPromptTemplate

prompt  = ChatPromptTemplate.from_messages([
    ("system","You a model to read and answer."),
    ("user","{input}")
])

In [ ]:
# here first chain is prompt and second one is llm
chain  = prompt | llm

In [ ]:
chain.invoke({"input":"define government intelligence?"})

AIMessage(content='Government intelligence refers to the information and insights gathered by government agencies through various means such as surveillance, espionage, analysis of open-source information, and other methods. This intelligence is used to inform government decision-making, protect national security, and support law enforcement efforts. Government intelligence agencies work to collect, analyze, and disseminate information on threats to the country, both domestically and internationally.', response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 23, 'total_tokens': 98}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_fa89f7a861', 'finish_reason': 'stop', 'logprobs': None})

In [ ]:
chain.invoke({"input":"define government intelligence?"}).content

'Government intelligence refers to the information and insights gathered by government agencies through various means such as surveillance, espionage, analysis of open-source information, and other methods. This intelligence is used to inform government decision-making, protect national security, and support law enforcement efforts. Government intelligence agencies work to collect, analyze, and disseminate information on threats to the country, both domestically and internationally.'

In [ ]:
from langchain_core.output_parsers import StrOutputParser
#third chain
output_str_parser = StrOutputParser()

In [ ]:
# let create a new chain
chain = prompt | llm | output_str_parser

In [ ]:
query="define government intelligence"
x=chain.invoke({"input":query})
x

'Government intelligence refers to the information and analysis gathered by government agencies to support decision-making and national security efforts. This intelligence is typically collected through various means, such as espionage, surveillance, and analysis of open-source information. Government intelligence is used to assess threats, inform policy decisions, and protect the interests of the state.'

In [ ]:
query="what was my input ? "
x=chain.invoke({"input":query})
x

"I'm sorry, but I don't have access to your previous input. How can I assist you today?"

In [ ]:
from langchain import PromptTemplate

from langchain.chains import RetrievalQA

from langchain.memory import ConversationBufferMemory

In [ ]:
template = """
Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the question:
------
<ctx>
{context}
</ctx>
------
<hs>
{history}
</hs>
------
{question}
Answer:
"""
prompt = PromptTemplate(
    input_variables=["history", "context", "question"],
    template=template,
)

In [ ]:
memory2 = ConversationBufferMemory(
    memory_key="history",
    input_key="question"
)

retrieval_chain = RetrievalQA.from_chain_type(llm,
                                              chain_type='stuff',
                                              retriever = db.as_retriever(),
                                              chain_type_kwargs={
                                                  "prompt": prompt,
                                                  "memory": memory2
                                              })

In [ ]:
q1="define management?"
q2="what are functions of management?"
q3="describe 14 principles of management by henri fayol?"
q4="what are principles of scientific management?"
q5="explain hygiene factors?"
q6="list the leadership styles?"
q7="who was Frederick Winslow Taylor?"
q8="list the features of scientific management?"
q9="how is management different from administration?"
q10="what defination of management was given by Peter F Drucker?"

In [ ]:
retrieval_chain.invoke(q1)

{'query': 'define management?',
 'result': 'Management can be defined as the art of getting things done through and with people in formally organized groups. It involves forecasting, planning, organizing, commanding, coordinating, and controlling resources to achieve predetermined goals. Management is also concerned with the systematic organization of economic resources to make them productive.'}

In [ ]:
memory2.load_memory_variables({})

{'history': 'Human: define management?\nAI: Management can be defined as the art of getting things done through and with people in formally organized groups. It involves forecasting, planning, organizing, commanding, coordinating, and controlling resources to achieve predetermined goals. Management is also concerned with the systematic organization of economic resources to make them productive.'}

In [ ]:
print(memory2.buffer)

Human: define management?
AI: Management can be defined as the art of getting things done through and with people in formally organized groups. It involves forecasting, planning, organizing, commanding, coordinating, and controlling resources to achieve predetermined goals. Management is also concerned with the systematic organization of economic resources to make them productive.


In [ ]:
retrieval_chain.invoke(q2)

{'query': 'what are functions of management?',
 'result': 'The functions of management include planning, organizing, staffing, directing, coordinating, and controlling. These functions are overlapping in nature and highly inseparable, as each function blends into the other and affects the performance of others.'}

In [ ]:
retrieval_chain.invoke(q3)

{'query': 'describe 14 principles of management by henri fayol?',
 'result': "Henri Fayol's 14 principles of management are as follows:\n\n1. Division of Labor: Specialization of jobs to increase efficiency.\n2. Authority and Responsibility: Managers must have the authority to give orders, but also the responsibility to ensure they are carried out.\n3. Discipline: Employees must respect authority and follow rules and regulations.\n4. Unity of Command: Each employee should receive orders from only one superior.\n5. Unity of Direction: One plan of action to guide the organization towards a common goal.\n6. Subordination of Individual Interests to the General Interest: The interests of the organization should come before individual interests.\n7. Remuneration: Fair compensation for employees based on their contributions.\n8. Centralization: The degree to which authority is concentrated at the top level.\n9. Scalar Chain: A clear line of authority from top management to the lowest ranks.\n

In [ ]:
retrieval_chain.invoke(q4)

{'query': 'what are principles of scientific management?',
 'result': 'The principles of scientific management, as developed by Frederick Winslow Taylor, include:\n\n1. Replacing rule of thumb with science\n2. Harmony in group action\n3. Co-operation\n4. Maximum output\n5. Development of workers\n\nIn terms of the general approach, the principles also involve:\n1. Shift in decision making from employees to managers\n2. Developing a standard method for performing each job\n3. Selecting workers with appropriate abilities for each job\n4. Training workers in the standard method previously developed\n\nThese principles emphasize the importance of scientific analysis and systematic management practices to improve efficiency and productivity in the workplace.'}

In [ ]:
retrieval_chain.invoke(q5)

{'query': 'explain hygiene factors?',
 'result': 'Hygiene factors are essential job factors that are necessary for the existence of motivation in the workplace. These factors do not directly lead to long-term positive satisfaction, but their absence or non-existence can result in dissatisfaction among employees. Hygiene factors, also known as dissatisfiers or maintenance factors, are required to prevent dissatisfaction. These factors include aspects such as pay, company policies, fringe benefits, physical working conditions, status, interpersonal relations, and job security. They are symbolized as fulfilling the physiological needs that individuals expect to be met in the workplace.'}

In [ ]:
retrieval_chain.invoke(q6)

{'query': 'list the leadership styles?',
 'result': '1. People-oriented leadership\n2. Servant leadership\n3. Task-oriented leadership\n4. Transactional leadership\n5. Transformational leadership\n6. Autocratic leadership\n7. Bureaucratic leadership\n8. Charismatic leadership\n9. Democratic leadership or Participative leadership\n10. Laissez-Faire leadership'}

In [ ]:
retrieval_chain.invoke(q7)

{'query': 'who was Frederick Winslow Taylor?',
 'result': 'Frederick Winslow Taylor (1856-1915) was an American mechanical engineer and the developer of scientific management, also known as Taylorism. He is widely regarded as the father of scientific management and was one of the first management consultants. Taylor\'s work focused on analyzing and synthesizing workflows to improve labor productivity, with the core ideas of his theory first published in his monographs "Shop Management" (1905) and "The Principles of Scientific Management" (1911). His approach emphasized replacing traditional decision-making methods with precise procedures developed through careful study of individual work processes. Taylor\'s work had a significant impact on the field of management and industrial efficiency during the late 19th and early 20th centuries.'}

In [ ]:
retrieval_chain.invoke(q8)

{'query': 'list the features of scientific management?',
 'result': 'The features of scientific management include:\n\n1. Scientific task setting\n2. Importance of compensation for performance\n3. Specialization in every part of defined labor\n4. Importance of selection criteria by management\n5. Scientific approach to business management and process improvement\n6. Careful study of tasks and jobs\n7. Planning the task\n8. Scientific selection and training of workers\n9. Standardization\n10. Specialization in the administrative and organizational setup of the plant'}

In [ ]:
retrieval_chain.invoke(q9)

{'query': 'how is management different from administration?',
 'result': 'Management and administration differ in terms of their functions and usage/applicability. \n\nManagement involves the guidance, leadership, and control of efforts towards common goals, focusing on getting things done through and with people in formally organized groups. It includes functions such as planning, organizing, staffing, directing, coordinating, and controlling resources to achieve predetermined objectives.\n\nOn the other hand, administration is about overall determination of policies, setting major objectives, identifying general purposes, and laying down broad programs and projects. It deals with higher-level activities, laying down basic principles of the enterprise.\n\nIn summary, management is more focused on the operational aspects of achieving goals through people, while administration is concerned with the broader strategic decisions and principles of the organization.'}

In [ ]:
retrieval_chain.invoke(q10)

{'query': 'what defination of management was given by Peter F Drucker?',
 'result': 'Peter F Drucker defined management as "Management is concerned with the systematic organization of economic resources and its task is to make these resources productive."'}

In [ ]:
memory2.load_memory_variables({})

{'history': 'Human: define management?\nAI: Management can be defined as the art of getting things done through and with people in formally organized groups. It involves forecasting, planning, organizing, commanding, coordinating, and controlling resources to achieve predetermined goals. Management is also concerned with the systematic organization of economic resources to make them productive.\nHuman: what are functions of management?\nAI: The functions of management include planning, organizing, staffing, directing, coordinating, and controlling. These functions are overlapping in nature and highly inseparable, as each function blends into the other and affects the performance of others.\nHuman: describe 14 principles of management by henri fayol?\nAI: Henri Fayol\'s 14 principles of management are as follows:\n\n1. Division of Labor: Specialization of jobs to increase efficiency.\n2. Authority and Responsibility: Managers must have the authority to give orders, but also the responsi

In [ ]:
print(memory2.buffer)

Human: define management?
AI: Management can be defined as the art of getting things done through and with people in formally organized groups. It involves forecasting, planning, organizing, commanding, coordinating, and controlling resources to achieve predetermined goals. Management is also concerned with the systematic organization of economic resources to make them productive.
Human: what are functions of management?
AI: The functions of management include planning, organizing, staffing, directing, coordinating, and controlling. These functions are overlapping in nature and highly inseparable, as each function blends into the other and affects the performance of others.
Human: describe 14 principles of management by henri fayol?
AI: Henri Fayol's 14 principles of management are as follows:

1. Division of Labor: Specialization of jobs to increase efficiency.
2. Authority and Responsibility: Managers must have the authority to give orders, but also the responsibility to ensure they 

In [ ]:
q11="what was my last question?"
retrieval_chain.invoke(q11)

{'query': 'what was my last question?',
 'result': 'Your last question was "what defination of management was given by Peter F Drucker?"'}

In [ ]:
q12="can u count and list all the questions or queries i aksed u?"
retrieval_chain.invoke(q12)

{'query': 'can u count and list all the questions or queries i aksed u?',
 'result': '1. Human: define management?\n2. Human: what are functions of management?\n3. Human: describe 14 principles of management by henri fayol?\n4. Human: what are principles of scientific management?\n5. Human: explain hygiene factors?\n6. Human: list the leadership styles?\n7. Human: who was Frederick Winslow Taylor?\n8. Human: list the features of scientific management?\n9. Human: how is management different from administration?\n10. Human: what defination of management was given by Peter F Drucker?\n11. Human: what was my last question?'}

Conversation Buffer Window Memory

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
memory2 = ConversationBufferWindowMemory(k=2)
conversation_buf2 = ConversationChain(
    llm=llm,
    memory=memory2
)